In [2]:
import mlflow
from mlflow.models import ModelSignature
from mlflow.types.schema import Schema, ColSpec, ParamSchema, ParamSpec
import requests
import numpy as np

class MyModel(mlflow.pyfunc.PythonModel):
    def predict(self, ctx, model_input, params):
        
        person_name = model_input['person_name'][0] #string
        person_height = model_input['person_height'][0] #double
        person_weight = model_input['person_weight'][0] #float
        person_age = model_input['person_age'][0] #integer
        person_id = model_input['person_id'][0] #long 
        person_is_alive = model_input['person_is_alive'][0] #bool
        
        return {"personName": str(type(person_name)),
                "personHeight": str(type(person_height)),
                "personWeight": str(type(person_weight)),
                "personAge": str(type(person_age)),
                "personId": str(type(person_id)),
                "personIsAlive": str(type(person_is_alive))
               }

In [3]:
input_schema = Schema(
    [
        ColSpec("string", "person_name"),
        ColSpec("double", "person_height"),
        ColSpec("float", "person_weight"),
        ColSpec("integer", "person_age"),
        ColSpec("long", "person_id"),
        ColSpec("boolean", "person_is_alive")
    ]
)

signature = ModelSignature(inputs=input_schema)

mlflow.set_experiment("person_model")
with mlflow.start_run() as run:
    model_info = mlflow.pyfunc.log_model(
        python_model=MyModel(),
        artifact_path="person_model",
        signature=signature,
        registered_model_name="person_model" # Register the model
    )

client = mlflow.MlflowClient()
model_metadata = client.get_latest_versions("person_model", stages=["None"])
latest_model_version = model_metadata[0].version

2024/01/26 16:35:17 INFO mlflow.tracking.fluent: Experiment with name 'person_model' does not exist. Creating a new experiment.
Successfully registered model 'person_model'.
2024/01/26 16:35:21 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: person_model, version 1
Created version '1' of model 'person_model'.


In [6]:
mlflow.models.get_model_info(f"models:/person_model/{latest_model_version}").signature

inputs: 
  ['person_name': string, 'person_height': double, 'person_weight': float, 'person_age': integer, 'person_id': long, 'person_is_alive': boolean]
outputs: 
  None
params: 
  None

# Usage

In the following are two ways to use the registered models. 

## Loading the model

Using pyfunc.load_model you can test if your model is working well, but it doesn`t involves nothing of the model services feature.

In order to use it, change the version of model_uri to the latests version you deployed

In [5]:
model = mlflow.pyfunc.load_model(model_uri=f"models:/person_model/{latest_model_version}")
print(model.predict({"person_name": "John", 
                     "person_height": 1.8, 
                     "person_weight": np.array(80, dtype=np.float32), 
                     "person_age": np.array(30, dtype=np.int32), 
                     "person_id": 12312312312, 
                     "person_is_alive": [True]}))

{'personName': "<class 'str'>", 'personHeight': "<class 'numpy.float64'>", 'personWeight': "<class 'numpy.float32'>", 'personAge': "<class 'numpy.int32'>", 'personId': "<class 'numpy.int64'>", 'personIsAlive': "<class 'numpy.bool_'>"}


## Model services

Go to the published services page and create a model, run it and wait until the running state. Change the URL field below to the url of the published services.

In [5]:
# Model service 🤘

url = "https://10.137.137.5:5000"+"/invocations"
data = {
    "inputs":{
        "person_name": "John", 
         "person_height": 1.8, 
         "person_weight": [80.00], 
         "person_age": [30],
         "person_id": 12312312312, 
         "person_is_alive": [True]
    }
}

headers = {'Content-type': 'application/json'}
x = requests.post(url, json = data, verify=False, headers=headers)

print(x.text)

{"predictions": {"flowerName": "sunflower", "do_work": false}}


/opt/conda/lib/python3.10/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.137.137.5'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
